<a href="https://colab.research.google.com/github/HugoMHard/tweet_processing/blob/main/%C3%A9tiquettage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import nltk
from nltk import word_tokenize
nltk.download('punkt_tab')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import string
import re
from nltk.probability import FreqDist
from nltk.util import ngrams
# Téléchargement du modèle spaCy en ligne de commande (à exécuter une seule fois)
#!python -m spacy download fr_core_news_md
nlp = spacy.load('fr_core_news_md')
nltk.download('stopwords')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# On importe les tweetsen csv

bdd_L_link = "/content/drive/MyDrive/projet/Tweet2.csv"
bdd_D_link = "/content/drive/MyDrive/projet/Tweet1.csv"
Tweet_csv_L = pd.read_csv(bdd_L_link, on_bad_lines='skip', sep=';')
Tweet_csv_D = pd.read_csv(bdd_D_link, on_bad_lines='skip',sep=';')

In [ ]:
# On réindexe et supprimme les lignes

Tweet_csv_L=Tweet_csv_L[['data__id','data__text']]
Tweet_csv_D=Tweet_csv_D[['data__id','data__text']]

Tweet_csv_L=(Tweet_csv_L.dropna(subset='data__text')).reset_index(drop=False)
Tweet_csv_D=(Tweet_csv_D.dropna(subset='data__text')).reset_index(drop=False)
Tweet_csv_L=(Tweet_csv_L[Tweet_csv_L['data__text']!='data__text'])[['index','data__text']]
Tweet_csv_D=(Tweet_csv_D[Tweet_csv_D['data__text']!='data__text'])[['index','data__text']]
#print(Tweet_csv_D.head(10))

In [ ]:
# On créer des dictionnaire contenant des dictionnaires sur les tweets

dict_combine, dict_L, dict_D = {}, {}, {}
l = len(Tweet_csv_L)

for index, row in Tweet_csv_L.iterrows():  # Iterate using iterrows()
  reset_index = row['index']
  tweet_text = row['data__text']  # Directly access the text as a string
  dict_combine[reset_index] = {'text': tweet_text}
  dict_L[reset_index] = {'text': tweet_text}

for index, row in Tweet_csv_D.iterrows():  # Iterate using iterrows()
  tweet_text = row['data__text']  # Directly access the text as a string
  dict_combine[reset_index + l -1] = {'text': tweet_text} # Updated index for combine
  dict_D[index] = {'text': tweet_text}

On ajoute au dico le texte tokeniser et trier des stop_words

In [ ]:
url_pattern = re.compile(r'(https?://|www\.)\S+')
pattern=r'\w+|#[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|@[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|[^\w\s]+'
tokenizer = RegexpTokenizer(pattern)

def remove_urls(text):
    return url_pattern.sub('url', text)

def remove_punctuation(text_without_urls):
    global dict_combine
    without_punctuation = []
    text_tokenise = tokenizer.tokenize(text_without_urls)
    for word in text_tokenise:
      if word.isalpha() or (word.startswith('#') and len(word_tokenize(word))>1) or (word.startswith('@') and len(word_tokenize(word))>1):
      # We import and use the stopwords from NLTK library
        if word.lower() not in stopwords.words('french'):
          without_punctuation.append(word.lower())
    dict_combine[key]['text_tokenise'] = without_punctuation
    return without

for key in dict_combine.keys():

  text=dict_combine[key]['text']
  text_without_urls = remove_urls(text)

  # Personnalisation du tokenizer pour garder les # ou @ avec le mot

  text_tokenise = tokenizer.tokenize(text_without_urls)
  without_punctuation = []

  # On tokenize en enlevant les stop_words en gardant les mentions et tweets
  for word in text_tokenise:
    if word.isalpha() or (word.startswith('#') and len(word_tokenize(word))>1) or (word.startswith('@') and len(word_tokenize(word))>1):
      # We import and use the stopwords from NLTK library
      if word.lower() not in stopwords.words('french'):
        without_punctuation.append(word.lower())
  dict_combine[key]['text_tokenise'] = without_punctuation

In [ ]:
def lemmatize_tokens(phrase_tokenisee):
    """
    Fonction pour lemmatiser les tokens d'une phrase tokenisée.

    :param phrase_tokenisee: Objet spaCy Doc contenant les tokens
    :return: Liste des mots lemmatisés
    """
    mots_lemmatises = []
    global mots_lem_g  # Accès à la liste globale pour stocker tous les mots lemmatisés

    j = 0
    while j < len(phrase_tokenisee):
        token = phrase_tokenisee[j]

        if token.text.startswith('#') and j + 1 < len(phrase_tokenisee):
            # Combinaison du hashtag avec le token suivant
            combined_text = token.text + phrase_tokenisee[j + 1].text
            combined_token = nlp(combined_text)
            mots_lemmatises.append(combined_token.text)
            mots_lem_g.append(combined_token.text)
            j += 2  # Passer au token après le suivant
        elif token.text.startswith('@'):
            # Lemmatisation des mentions
            mots_lemmatises.append(token.lemma_)
            mots_lem_g.append(token.lemma_)
            j += 1
        else :
            # Lemmatisation des autres mots
            mots_lemmatises.append(token.lemma_)
            mots_lem_g.append(token.lemma_)
            j += 1

    return mots_lemmatises

In [ ]:
# Liste globale pour stocker tous les mots lemmatisés
mots_lem_g = []

# Parcours des clés du dictionnaire
for index, value in dict_combine.items():
    # Création de l'objet spaCy à partir de la liste 'text_tokenise'
    phrase_tokenisee = nlp(" ".join(value['text_tokenise']))

    # Appel de la fonction pour lemmatiser les tokens
    value['mots_lemmatises'] = lemmatize_tokens(phrase_tokenisee)


In [ ]:
#afficher les k n-grammes les plus communs dans le texte, ici les 100 premiers trigrammes
n=3
k=100
ngram = list(nltk.ngrams(mots_lem_g,n))
fdist = FreqDist(ngram)
print(fdist.most_common(k))

[(('url', 'url', 'rt'), 185), (('loi', 'asil', 'immigration'), 65), (('url', 'rt', '@jlmelenchon'), 60), (('url', 'rt', '@estherbenbassa'), 59), (('#migrants', 'url', 'rt'), 46), (('url', 'rt', '@brnomorel'), 46), (('url', 'rt', '@euroecolos'), 45), (('url', 'rt', '@dversini'), 43), (('migrant', 'url', 'rt'), 39), (('loi', '#asile', 'immigration'), 31), (('rt', '@estherbenbassa', '#migrants'), 31), (('url', 'rt', '@damiencareme'), 30), (('url', 'rt', '@sandraregol'), 29), (('liberté', 'circulation', 'installation'), 28), (('politique', 'anti', 'migrant'), 25), (('url', 'rt', '@lcp'), 24), (('gt', 'gt', 'url'), 24), (('url', 'rt', '@faureolivier'), 24), (('accueil', 'digne', 'migrant'), 23), (('url', 'rt', '@mdm_france'), 23), (('url', 'rt', '@ericcoquerel'), 22), (('url', 'rt', '@manonaubryfr'), 21), (('centre', 'rétention', 'administratif'), 20), (('politique', 'migratoire', 'gouvernement'), 19), (('projet', 'loi', 'asil'), 19), (('url', 'rt', '@benoithamon'), 19), (('#lemissionpoliti

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

for key, value in dict_combine.items():
    tweet = value['text'].iloc[0]  # Access the tweet text
    tokens = word_tokenize(tweet)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    value['lemmas'] = lemmas # Add the lemmas to the dictionary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


AttributeError: 'str' object has no attribute 'iloc'

In [ ]:
#!python -m spacy download fr_core_news_md
import spacy
import nltk
from nltk import FreqDist
#spacy.load('fr_core_news_md') # No need to call spacy.load twice
# On va lématiser les mots
nlp = spacy.load('fr_core_news_md')
mots_lem_g=[]
for index in dict_combine.keys():
  mots_lemmatises=[]
  # Accessing the correct key 'text_tokenise'
  phrase_de_mots_tokenise=nlp(" ".join(dict_combine[index]['text_tokenise']))
  contenant = {}
  for i,w in enumerate(phrase_de_mots_tokenise):
    contenant[i]=w
    j=0
  while (j)!=len(phrase_de_mots_tokenise):
    w=contenant[j]
    if  w.text.startswith('#') and (j+1!=len(phrase_de_mots_tokenise)):
      w=w.text+phrase_de_mots_tokenise[j+1].text
      w=nlp(w)  # Get the first token from the processed text
      mots_lemmatises.append(w) # Append lemma of combined token
      mots_lem_g.append(w)
      j+=2
    elif w.text.startswith('@') and (j+1!=len(phrase_de_mots_tokenise)):
      mots_lemmatises.append(w.lemma_)
      mots_lem_g.append(w.lemma_)
      j+=1
    else:
      mots_lemmatises.append(w.lemma_)
      mots_lem_g.append(w.lemma_)
      j+=1

  dict_combine[index]['mots_lemmatises']=mots_lemmatises

In [ ]:
url_pattern = re.compile(r'(https?://|www\.)\S+')
pattern=r'\w+|#[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|@[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|[^\w\s]+'
tokenizer = RegexpTokenizer(pattern)
for key in dict_combine.keys():

  text=dict_combine[key]['text']
  text_without_urls = url_pattern.sub('url', text) # On enlève les urls du code

  # Personnalisation du tokenizer pour garder les # ou @ avec le mot

  text_tokenise = tokenizer.tokenize(text_without_urls)
  without_punctuation = []

  # On tokenize en enlevant les stop_words en gardant les mentions et tweets
  for word in text_tokenise:
    if word.isalpha() or (word.startswith('#') and len(word_tokenize(word))>1) or (word.startswith('@') and len(word_tokenize(word))>1):
      if word.lower() not in stopwords.words('french'):
        without_punctuation.append(word.lower())
  dict_combine[key]['text_tokenise'] = without_punctuation

In [ ]:
#!python -m spacy download fr_core_news_md
import spacy
import nltk
from nltk import FreqDist
#spacy.load('fr_core_news_md') # No need to call spacy.load twice
# On va lématiser les mots
nlp = spacy.load('fr_core_news_md')
mots_lem_g=[]
for index in dict_combine.keys():
  mots_lemmatises=[]
  # Accessing the correct key 'text_tokenise'
  phrase_de_mots_tokenise=nlp(" ".join(dict_combine[index]['text_tokenise']))
  for i,w in enumerate(phrase_de_mots_tokenise):
    if (w.text.startswith('@') or w.text.startswith('#')) and (i+1!=len(phrase_de_mots_tokenise)):
      w=w.text+phrase_de_mots_tokenise[i+1].text
      combined_token = nlp(w)[0]  # Get the first token from the processed text
      mots_lemmatises.append(combined_token.lemma_) # Append lemma of combined token
      mots_lem_g.append(combined_token.lemma_)
    else:
      mots_lemmatises.append(w.lemma_)
      mots_lem_g.append(w.lemma_)
  dict_combine[index]['mots_lemmatises']=mots_lemmatises